In [1]:
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
import qdrant_client
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
QDRANT_URL = os.getenv('QDRANT_URL')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
ARES_API_KEY = os.getenv('ARES_API_KEY')
VECTOR_DB_COLLECTION = 'traversal'

In [19]:
OPENAI_API_KEY = os.getenv('open_ai_key')


In [3]:
ARES_API_KEY

'ares_6bef1fc88e7c0daa492494b7a863936dd8326dc8d37152599b5b384b6a907a94'

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
qdrant_client_obj = qdrant_client.QdrantClient(
        url=QDRANT_URL,
        prefer_grpc=True,
        api_key=QDRANT_API_KEY)
qdrant = Qdrant(qdrant_client_obj, VECTOR_DB_COLLECTION, embeddings)

In [6]:
retriever = qdrant.as_retriever(search_type="mmr") # qdrant is used as retriever to make a chain

## Step 1: Semantic Search with Qdrant

In [7]:
query = "describe Romance Istanbul Hotel"
res = retriever.get_relevant_documents(query)[0].page_content
print(res)

hotel_name: Romance Istanbul Hotel
hotel_description: Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.
review_title: Very nice visit to Turkey
review_text:

In [8]:
import json
import re

# Define a regex pattern to match key-value pairs
pattern = r'(?P<key>.*?):\s*(?P<value>.*)'

# Initialize an empty dictionary to store key-value pairs
json_data = {}

# Iterate over matches of the pattern in the input string
for match in re.finditer(pattern, res):
    key = match.group('key').strip()
    value = match.group('value').strip()
    json_data[key] = value

# Convert the dictionary to JSON
json_string = json.dumps(json_data, indent=4)

# Print the JSON string
print(json_string)

{
    "hotel_name": "Romance Istanbul Hotel",
    "hotel_description": "Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city\u2019s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.",
    "review_title": "Very n

In [9]:
print(json_data)

{'hotel_name': 'Romance Istanbul Hotel', 'hotel_description': 'Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room.', 'review_title': 'Very nice visit to Turke

In [10]:
hotel_name = json_data['hotel_name']

In [11]:
hotel_name

'Romance Istanbul Hotel'

## Step 2: traversaal

In [12]:
import requests

url = "https://api-ares.traversaal.ai/live/predict"

payload = { "query": ["taco spots around" + hotel_name] }
headers = {
  "x-api-key": ARES_API_KEY,
  "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [13]:
response

<Response [200]>

'{"data":{"response_text":"There are several taco spots near the Romance Istanbul Hotel. Here are some options:\\n\\n1. Ranchero - Located at Plaj Yolu Sok.\\n2. Los Altos - Located at Yeni Carsi Caddesi Tomtom Mahallesi No:38.\\n3. Tacofit - Located at Nisbetiye Cad. No:42/A, Beşiktaş.\\n4. El Rey del Taco - Located at Carr. Tulum - Cancún, Plaza Antares, Playa del Carmen.\\n5. El Fogon - Located at Calle 14 Nte Bis, Gonzalo Guerrero, Playa del Carmen.\\n6. Taco Paco - Located at Carr. Tulum - Cancún, Plaza Antares, Playa del Carmen.\\n7. Taquería Honorio - Located at Tulum.\\n8. Don Sirloin - Located at Plaza Point 115, Av. Ptempich, Playa del Carmen.\\n9. Carboncitos - Located at Calle 4 Nte S/N, Centro, Playa del Carmen.\\n10. Señor Taco - Located at 10 Avenida Nte. 221, Gonzalo Guerrero, Playa del Carmen.\\n\\nPlease note that the exact distance and availability may vary, so it\'s recommended to check the addresses and operating hours before visiting.","web_url":["https://foursquare.com/top-places/istanbul/best-places-tacos","https://www.yelp.com/biz/tacofit-istanbul","https://www.romanceistanbulhotel.com/","https://restaurantguru.com/tacos-Istanbul-m6523","https://www.yelp.com/search?cflt=mexican&find_loc=Istanbul","https://www.romanceistanbulhotel.com/romance-hotel-dining/","https://www.grubhub.com/restaurant/istanbul-gyro--tacos-halal-8409-se-45th-ave-portland/4179728","https://guide.michelin.com/us/en/istanbul-province/istanbul/restaurant/azur","https://www.timeout.com/istanbul/things-to-do/kadikoy-moda-guide","https://www.barcelo.com/pinandtravel/en/best-riviera-maya-tacos/","https://www.tiktok.com/@nyctravelgal/video/7237516991509138731","https://www.findmeglutenfree.com/tr/istanbul/breakfast","https://dametraveler.com/restaurants-we-love-bar-otaku/","https://www.willflyforfood.net/istanbul-restaurants/","https://www.gimmesomeoven.com/life/alis-guide-to-mexico-city/","https://johnnyafrica.com/mexico-city-tacos-tour-itinerary/"]}}'


In [14]:
json_data = json.loads(response.text)

# Access the 'response_text' field
response_text = json_data['data']['response_text']

# Print the 'response_text'
print(response_text)

There are several taco spots near the Romance Istanbul Hotel. Here are some options:

1. Ranchero - Located at Plaj Yolu Sok.
2. Los Altos - Located at Yeni Carsi Caddesi Tomtom Mahallesi No:38.
3. Tacofit - Located at Nisbetiye Cad. No:42/A, Beşiktaş.
4. El Rey del Taco - Located at Carr. Tulum - Cancún, Plaza Antares, Playa del Carmen.
5. El Fogon - Located at Calle 14 Nte Bis, Gonzalo Guerrero, Playa del Carmen.
6. Taco Paco - Located at Carr. Tulum - Cancún, Plaza Antares, Playa del Carmen.
7. Taquería Honorio - Located at Tulum.
8. Don Sirloin - Located at Plaza Point 115, Av. Ptempich, Playa del Carmen.
9. Carboncitos - Located at Calle 4 Nte S/N, Centro, Playa del Carmen.
10. Señor Taco - Located at 10 Avenida Nte. 221, Gonzalo Guerrero, Playa del Carmen.

Please note that the exact distance and availability may vary, so it's recommended to check the addresses and operating hours before visiting.


In [15]:
json_data = json.loads(response.text)

# Access the 'response_text' field
response_text = json_data['data']['web_url']

# Print the 'response_text'
print(response_text)

['https://foursquare.com/top-places/istanbul/best-places-tacos', 'https://www.yelp.com/biz/tacofit-istanbul', 'https://www.romanceistanbulhotel.com/', 'https://restaurantguru.com/tacos-Istanbul-m6523', 'https://www.yelp.com/search?cflt=mexican&find_loc=Istanbul', 'https://www.romanceistanbulhotel.com/romance-hotel-dining/', 'https://www.grubhub.com/restaurant/istanbul-gyro--tacos-halal-8409-se-45th-ave-portland/4179728', 'https://guide.michelin.com/us/en/istanbul-province/istanbul/restaurant/azur', 'https://www.timeout.com/istanbul/things-to-do/kadikoy-moda-guide', 'https://www.barcelo.com/pinandtravel/en/best-riviera-maya-tacos/', 'https://www.tiktok.com/@nyctravelgal/video/7237516991509138731', 'https://www.findmeglutenfree.com/tr/istanbul/breakfast', 'https://dametraveler.com/restaurants-we-love-bar-otaku/', 'https://www.willflyforfood.net/istanbul-restaurants/', 'https://www.gimmesomeoven.com/life/alis-guide-to-mexico-city/', 'https://johnnyafrica.com/mexico-city-tacos-tour-itinera

### Step 3 OpenAI

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.chat import ChatPromptTemplate

In [21]:
template = """
Answer the user's questions: {question} based on the below context:\n\n{context} remember that you are awesome hotel advisor
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2,api_key=OPENAI_API_KEY)

rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
print(rag_chain.invoke(query))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Based on the provided information, Romance Istanbul Hotel is a boutique hotel located in Istanbul, Turkey. It has 39 elegantly furnished rooms that blend modern lifestyle with traditional Ottoman touches. The hotel is situated at the intersection of the old city's most important part, making it convenient for exploring historical sites like the Hagia Sophia and Topkapi Palace.

The hotel offers various room types, including suite rooms, luxury rooms with a terrace, deluxe rooms, and city rooms. Each room comes with a private bathroom, shower, satellite TV, and free wifi connection. Guests have praised the friendly and welcoming staff, as well as the hotel's cozy decoration and relaxing amenities like the hamam.

Overall, Romance Istanbul Hotel has received positive reviews for its prime location, excellent service, and comfortable accommodations, making it a highly recommended choice for visitors to Istanbul.


Romance Istanbul Hotel is a boutique hotel located in Istanbul with 39 elegantly furnished rooms that blend modern lifestyle with traditional Ottoman touches. Situated at the intersection of the old city's most important part, the hotel offers exceptional design, world-renowned Turkish hospitality, and personalized service. Each room features a private bathroom, shower, satellite TV, and free wifi connection, with sizes ranging from 20 m2 to 45 m2. The hotel includes various room types such as suite rooms, luxury rooms with terrace, deluxe rooms, and city rooms. Guests have praised the hotel's great location near historical sites, friendly staff, delicious breakfast, and clean rooms. Overall, Romance Istanbul Hotel offers a comfortable and convenient stay for visitors to Istanbul.